at first try to work with images in cv2

In [1]:
import os


WORK_DIR_PATH = os.getcwd()
DATA_DIR = 'data/label_annotated_train'
DATA_PATH = os.path.join(WORK_DIR_PATH, DATA_DIR)
print(f'Work in {DATA_PATH}')
DATA_FILES_PATHS = [os.path.join(DATA_PATH, name) for name in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, name))]
DATA_FOLDERS_PATHS = [os.path.join(DATA_PATH, name) for name in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, name))]
# for folder in DATA_FOLDERS_PATHS:         
#     print(folder, os.listdir(folder))
#print(f'Files in folder {DATA_FILES_PATHS}')


Work in /Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train


In [2]:
# convert annotated files (xml) in folder to txt
# from https://blog.paperspace.com/train-yolov5-custom-data/
import xml.etree.ElementTree as ET
import cv2

class extractors_xml2txt():
    """give path to data folder and writes .txt files from .xml in it"""
    
    def __init__(self, DATA_FOLDERS_PATHS=DATA_FOLDERS_PATHS):
        self.folders = DATA_FOLDERS_PATHS
        # Dictionary that maps class names to IDs
        self.class_name_to_id_mapping = {"doc_type": 0, "doc_name": 1, "doc_date": 2}
        #self.classes = {0: "doc_type", 1: "doc_name", 2: "doc_date"}
        self.classes = dict(zip(self.class_name_to_id_mapping.values(), self.class_name_to_id_mapping.keys()))

        

    def find_and_replace(self):
        try:
            self.find_xml_file()
            return True
        except Exception as e:
            print(f'Raised exception: {e}')
            return False
        
    def find_xml_file(self):
        for folder in self.folders:
            for name in os.listdir(folder):
                file_path = os.path.join(folder, name)
                if os.path.isfile(file_path):
                    file_ext = name.split('.')[-1]
                    if file_ext == 'xml':
                        extracted_data = self.extract_xml_data(file_path=file_path)
                        self.write2txt(from_file=file_path, extracted_data=extracted_data)
                    
    def extract_xml_data(self, file_path):
        """ extracted data is {'bboxes': [
        {'class': 'doc_type', 'xmin': 1468, 'ymin': 1335, 'xmax': 1568, 'ymax': 2024}, 
        {'class': 'doc_name', 'xmin': 1500, 'ymin': 2095, 'xmax': 1570, 'ymax': 2476}, 
        {'class': 'doc_date', 'xmin': 1492, 'ymin': 2600, 'xmax': 1578, 'ymax': 2916}], 
        'filename': 'bop.3.jpg', 'image_size': (3328, 4677, 1)}"""
        #return dictionary
        root = ET.parse(file_path).getroot()
        # Initialise the info dict 
        info_dict = {}
        info_dict['bboxes'] = []
        # Parse the XML Tree
        for elem in root:
            # Get the file name 
            if elem.tag == "filename":
                info_dict['filename'] = elem.text

            # Get the image size
            elif elem.tag == "size":
                image_size = []
                for subelem in elem:
                    image_size.append(int(subelem.text))
                info_dict['image_size'] = tuple(image_size)

            # Get details of the bounding box 
            elif elem.tag == "object":
                bbox = {}
                for subelem in elem:
                    if subelem.tag == "name":
                        bbox["class"] = subelem.text

                    elif subelem.tag == "bndbox":
                        for subsubelem in subelem:
                            bbox[subsubelem.tag] = int(subsubelem.text)            
                info_dict['bboxes'].append(bbox)
        return info_dict              
    
    def name_convertor_xml2txt(self, from_file):
        file_folders = from_file.split('/')[0:-1]
        file_folder_path = '/'.join(file_folders)
        file_full_name = from_file.split('/')[-1]
        file_name = file_full_name.split('.')[0:-1]
        file_name_str = '.'.join(file_name)
        file_ext = file_full_name.split('.')[-1]
        new_txt_file_path = str(file_folder_path) + '/' + str(file_name_str) + '.txt'
        jpg_file_path = str(file_folder_path) + '/' + str(file_name_str) + '.jpg'
        return [new_txt_file_path, jpg_file_path]
    
    def write2txt(self,from_file, extracted_data):
        print_buffer = []
        for b in extracted_data["bboxes"]:
            try:
                class_id = self.class_name_to_id_mapping[b["class"]]
            except KeyError:
                print(f"Invalid Class {b['class']}. Must be one from ", self.class_name_to_id_mapping.keys())

            # Transform the bbox co-ordinates as per the format required by YOLO v5
            b_center_x = (b["xmin"] + b["xmax"]) / 2 
            b_center_y = (b["ymin"] + b["ymax"]) / 2
            b_width    = (b["xmax"] - b["xmin"])
            b_height   = (b["ymax"] - b["ymin"])
            #print(self.name_convertor_xml2txt(from_file))
            
            # Normalise the co-ordinates by the dimensions of the image
            image_w, image_h, image_c = extracted_data["image_size"]  
            b_center_x /= image_w 
            b_center_y /= image_h 
            b_width    /= image_w 
            b_height   /= image_h

            #Write the bbox details to the file 
            print_buffer.append(f"{class_id} {b_center_x:.3f} {b_center_y:.3f} {b_width:.3f} {b_height:.3f}")
        
        # Name of the file which we have to save 
        #save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
        save_file_name = self.name_convertor_xml2txt(from_file)[0]
        # Save the annotation to disk
        with open(save_file_name, "w") as f:
            f.write("\n".join(print_buffer))
        #print("\n".join(print_buffer), file= open(save_file_name, "w"))
    
    
    def check_txt_annotation(self):
        #show images in window with boxes from txt file
        for folder in self.folders:
            for name in os.listdir(folder):
                file_path = os.path.join(folder, name)
                if os.path.isfile(file_path):
                    file_ext = name.split('.')[-1]
                    if file_ext == 'txt':
                        with open(file_path, "r") as file:
                            annotation_list = file.read().split("\n")
                            annotation_list = [x.split(" ") for x in annotation_list]
                            annotation_list = [[float(y) for y in x ] for x in annotation_list]
                            # image
                            jpg_file = self.name_convertor_xml2txt(file_path)[1]
                            img = cv2.imread(jpg_file)
                            height, width, channels = img.shape
                            for box in annotation_list:
                                x1 = int((box[1]-box[3]/2)*width)
                                x2 = int((box[1]+box[3]/2)*width)
                                y1 = int((box[2]-box[4]/2)*height)
                                y2 = int((box[2]+box[4]/2)*height)
                                cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=5, shift = 0)
                                obj_class = self.classes[(int(box[0]))]
                                cv2.putText(img, obj_class, (x1,y1), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
                                jpg_file_name = 'File: ' + jpg_file
                                cv2.putText(img, jpg_file_name, (50,50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 3)
                                
                            window_name = 'press q for quit'
                            cv2.imshow(window_name,img)
                            res = cv2.waitKey(0)
                            #press q for stop checking
                            if res in [113, 27] : 
                                cv2.destroyWindow(window_name)
                                cv2.waitKey(1)
                                return
                            #show next image
                            cv2.destroyWindow(window_name)
                            cv2.waitKey(1)
        
    def check_xml_annotation(self):
        #show images in window with boxes from xml file
        for folder in self.folders:
            for name in os.listdir(folder):
                file_path = os.path.join(folder, name)
                if os.path.isfile(file_path):
                    file_ext = name.split('.')[-1]
                    if file_ext == 'xml':
                        xml_dict = self.extract_xml_data(file_path)
                        jpg_file = self.name_convertor_xml2txt(file_path)[1]
                        img = cv2.imread(jpg_file)
                        for box in xml_dict['bboxes']:
                            obj_class = box['class']
                            x1 = box['xmin']
                            y1 = box['ymin']
                            x2 = box['xmax']
                            y2 = box['ymax']
                            
                            
                            cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=5, shift = 0)
                            cv2.putText(img, obj_class, (x1,y1), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
                            jpg_file_name = 'File: ' + jpg_file
                            cv2.putText(img, jpg_file_name, (50,50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 3)
                                
                        window_name = 'press q for quit'
                        cv2.imshow(window_name,img)
                        res = cv2.waitKey(0)
                        #press q for stop checking
                        if res in [113, 27] : 
                            cv2.destroyWindow(window_name)
                            cv2.waitKey(1)
                            return
                        #show next image
                        cv2.destroyWindow(window_name)
                        cv2.waitKey(1)
        
    
# extr = extractors_xml2txt(DATA_FOLDERS_PATHS) 
# print(extr.find_and_replace())
# extr.check_xml_annotation()



#test_file = '/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test/bop.3.xml'
#file_data_dict = extr.extract_data(test_file)


In [3]:
import cv2
test_file = '/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test/bop.3.jpg'
img = cv2.imread(test_file)
cv2.imshow('jpg_file',img)
res = cv2.waitKey(0)
print(res)
cv2.destroyWindow('jpg_file')
cv2.waitKey(1)

27


-1

In [4]:
#test in cv2
import cv2
test_jpg_file = '/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test/bop.3.jpg'
img = cv2.imread(test_jpg_file)
window_name = 'press q for quit'
x1 = 1492
x2 = 1578
y1 = 2600
y2 = 2916
pt1 = (x1,y1)
pt2 = (x2,y2)
crop_img = img[y1:y2, x1:x2]
cv2.rectangle(img, pt1, pt2, color=(0, 255, 0), thickness=5, shift = 0)
cv2.imshow(window_name,crop_img)
cv2.waitKey(0)
cv2.destroyWindow(window_name)
cv2.waitKey(1)
write_to_file = '/Users/johnlennon/My Drive/Python/Pdf/data/label_annotated_train/test/bop.cropped.jpg'
cv2.imwrite(write_to_file, crop_img)

True